In [1]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

# تنظیمات اتصال به SQL Server با Windows Authentication
params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-NI8V1U6\\SQL2022;"
    "DATABASE=OlistStaging;"
    "Trusted_Connection=yes;"
)

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
print("✅ اتصال با موفقیت برقرار شد.")

✅ اتصال با موفقیت برقرار شد.


In [2]:
def load_csv_to_sql(file_path, table_name, engine, if_exists='append'):
    df = pd.read_csv(file_path)
    df.to_sql(table_name, con=engine, index=False, if_exists=if_exists)
    print(f"📥 داده‌ها به جدول '{table_name}' منتقل شدند. (تعداد ردیف: {len(df)})")


In [7]:
import pandas as pd

# 1. خواندن داده‌ها از فایل CSV
df = pd.read_csv('../data/processed/olist_order_reviews_cleaned.csv')

# 2. حذف رکوردهای تکراری در فایل
df = df.drop_duplicates(subset='review_id')

# 3. استخراج review_id‌های موجود در دیتابیس
existing_ids = pd.read_sql("SELECT review_id FROM Reviews", con=engine)

# 4. فیلتر کردن فقط review_id‌های جدید
new_df = df[~df['review_id'].isin(existing_ids['review_id'])]

print(f"✅ تعداد رکوردهای جدید برای درج: {len(new_df)}")

# 5. درج فقط رکوردهای جدید در جدول
new_df.to_sql('Reviews', con=engine, index=False, if_exists='append')
print("📥 داده‌های جدید با موفقیت وارد شدند.")


✅ تعداد رکوردهای جدید برای درج: 95121
📥 داده‌های جدید با موفقیت وارد شدند.


In [9]:
def load_csv_to_sql(file_path, table_name, engine, if_exists='append', rename_columns=None):
    df = pd.read_csv(file_path)

    if rename_columns:
        df.rename(columns=rename_columns, inplace=True)

    df.to_sql(table_name, con=engine, index=False, if_exists=if_exists)
    print(f"📥 داده‌ها به جدول '{table_name}' منتقل شدند. (تعداد ردیف: {len(df)})")


In [10]:
     load_csv_to_sql(
    base_path + 'olist_products_cleaned.csv',
    'Products',
    engine,
    rename_columns={
        'product_name_lenght': 'product_name_length',
        'product_description_lenght': 'product_description_length'
    }
)

📥 داده‌ها به جدول 'Products' منتقل شدند. (تعداد ردیف: 32341)


In [11]:
load_csv_to_sql(base_path + 'olist_sellers_cleaned.csv', 'Sellers', engine)

📥 داده‌ها به جدول 'Sellers' منتقل شدند. (تعداد ردیف: 3095)


In [12]:
load_csv_to_sql(base_path + 'olist_orders_cleaned.csv', 'Orders', engine)

📥 داده‌ها به جدول 'Orders' منتقل شدند. (تعداد ردیف: 99441)
